In [1]:
import os 
import numpy as np
from PIL import Image,ImageTk
from io import BytesIO
import requests
from tkinter import Tk, Label,Canvas,Entry,Button,NW
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D,Flatten,Dense
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.preprocessing import image as image_utils

In [2]:
model =Sequential()

model.add(Conv2D(32,(3,3),input_shape=(128,128,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(128,activation="relu"))
model.add(Dense(2,activation="sigmoid"))

In [3]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [4]:
train_datagen=ImageDataGenerator(rescale=1.0/255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [5]:
test_dategen=ImageDataGenerator(rescale=1.0/255)

In [6]:
training_datasets=train_datagen.flow_from_directory("datasetcatdog/training_set",target_size=(128,128),batch_size=32,class_mode="categorical")
testing_dategen=test_dategen.flow_from_directory("datasetcatdog/test_set",target_size=(128,128),batch_size=32,class_mode="categorical")

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [7]:
model.fit(training_datasets,steps_per_epoch=4002/32, epochs=5,validation_data=testing_dategen,validation_steps=1012/32)

Epoch 1/5
125/125 [==============================] - 137s 1s/step - loss: 0.6924 - accuracy: 0.5682 - val_loss: 0.6673 - val_accuracy: 0.5801
Epoch 2/5
125/125 [==============================] - 151s 1s/step - loss: 0.6271 - accuracy: 0.6488 - val_loss: 0.5911 - val_accuracy: 0.6943
Epoch 3/5
125/125 [==============================] - 156s 1s/step - loss: 0.6049 - accuracy: 0.6634 - val_loss: 0.5765 - val_accuracy: 0.7041
Epoch 4/5
125/125 [==============================] - 165s 1s/step - loss: 0.5713 - accuracy: 0.6959 - val_loss: 0.5876 - val_accuracy: 0.6836
Epoch 5/5
125/125 [==============================] - 139s 1s/step - loss: 0.5424 - accuracy: 0.7324 - val_loss: 0.5931 - val_accuracy: 0.6719


In [8]:
model.save("datasetcatdog/model.h5")
model.save_weights("datasetcatdog/model_weights.h5")

In [9]:
image_width, image_height = 128, 128
model_path = "model/model.h5"
model_wight_path = "model/model_weights.h5"

final_model = load_model(model_path) 
final_model.load_weights(model_wight_path)

In [10]:
url=''
window=Tk()
window.title("Image Classifcation Using CNN")#
window.geometry("800x800")
label=Label(window,text="Please Enter your Custom URL",font=("Halvetica",16))
label.pack()

In [11]:
def Enter():
    global url
    label.configure()
    url=(User_input.get())
    print(url)
    
    response=requests.get(url)
    test_image=Image.open(BytesIO(response.content))
    put_image=test_image.resize((400,400))
    test_image=test_image.resize((128,128))
    
    img=ImageTk.PhotoImage(put_image)
    pic=Label(image=img)
    pic.pack()
    
    pic.image=img
    test_image=image_utils.img_to_array(test_image)
    test_image=np.expand_dims(test_image,axis=0)
    
    results=model.predict_on_batch(test_image)
    
    if results[0][0]==1:
        res="Dog"
    elif results[0][1]==1:
        res="Cat"
        
    output=Label(window,text=" Predicted Results: "+ res,font=("Halvetica",16))
    output.pack()

In [ ]:
User_input=Entry()
User_input.pack()
button=Button(window,text=" Detect Image : " ,font=("Halvetica",16),command=Enter)
button.pack()
window.mainloop()